In [1]:
# %pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/repo.html
# %pip install labml-nn

In [2]:
# %pip install pydantic

In [3]:
import dgl.sparse as dglsp
import torch
import torch.nn as nn
import torch.nn.functional as F


class LinearNeuralNetwork(nn.Module):
    def __init__(self, nfeat, nclass, bias=True):
        super(LinearNeuralNetwork, self).__init__()
        self.W = nn.Linear(nfeat, nclass, bias=bias)

    def forward(self, x):
        return self.W(x)


def symmetric_normalize_adjacency(graph):
    """Symmetric normalize graph adjacency matrix."""
    indices = torch.stack(graph.edges())
    n = graph.num_nodes()
    adj = dglsp.spmatrix(indices, shape=(n, n))
    deg_invsqrt = dglsp.diag(adj.sum(0)) ** -0.5
    return deg_invsqrt @ adj @ deg_invsqrt


def model_test(model, embeds):
    model.eval()
    with torch.no_grad():
        output = model(embeds)
        pred = output.argmax(dim=-1)
        test_mask, tv_mask = model.test_mask, model.tv_mask
        loss_tv = F.mse_loss(output[tv_mask], model.label_one_hot[tv_mask])
    accs = []
    for mask in [tv_mask, test_mask]:
        accs.append(
            float((pred[mask] == model.label[mask]).sum() / mask.sum()))
    return loss_tv.item(), accs[0], accs[1], pred

c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()
c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import argparse
import time
import torch.optim as optim
from dgl import AddSelfLoop
from dgl.data import CiteseerGraphDataset

parser = argparse.ArgumentParser()
args = parser.parse_args([])
args.dataset = "citeseer"
args.decline = 0.9
args.lr_sup = 0.001
args.lr_clf = 0.5
args.beta = 0.1
args.max_sim_rate = 0.995
args.max_patience = 2
args.device = torch.device("cpu")

In [5]:
!nvidia-smi

Sun Oct 20 17:42:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P0              13W /  50W |    121MiB /  4096MiB |     30%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
transform = AddSelfLoop()
data = CiteseerGraphDataset(transform=transform)
graph = data[0].to(args.device)

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [7]:
graph.has_edges_between([0], [1])

tensor([False])

In [8]:
def pagerank(net, weights={}, q=0.5, eps=0.01, maxIters=500, verbose=False, weightName='weight'):

    incomingTeleProb = {}
    prevVisitProb = {}
    currVisitProb = {}
    N = net.number_of_nodes()

    totWeight = sum([w for v, w in weights.items()])

    if totWeight == 0:
        incomingTeleProb = dict.fromkeys(net, 1.0 / N)
        prevVisitProb = incomingTeleProb.copy()
        currVisitProb = incomingTeleProb.copy()
    else:
        minPosWeight = 1.0
        for v, weight in weights.items():
            if weight == 0:
                continue
            minPosWeight = min(minPosWeight, 1.0 * weight / totWeight)

        smallWeight = minPosWeight / (10 ** 6)

        for v in net.nodes():
            weight = weights.get(v, 0.0)
            incomingTeleProb[v] = 1.0 * \
                (weight + smallWeight) / (totWeight + smallWeight * N)
        prevVisitProb = incomingTeleProb.copy()
        currVisitProb = incomingTeleProb.copy()

    outDeg = {}
    zeroDegNodes = set()
    for v in net.nodes():
        outDeg[v] = 1.0 * net.out_degree(v, weight=weightName)
        if outDeg[v] == 0:
            zeroDegNodes.add(v)

    iters = 0
    finished = False
    while not finished:
        iters += 1
        prevVisitProb = currVisitProb.copy()
        maxDiff = 0

        zSum = sum([prevVisitProb[x] for x in zeroDegNodes]) / N

        for v in net.nodes():
            eSum = 0
            for u in net.predecessors(v):
                # Handle missing 'weight' attribute by assuming it's 1.0
                w_uv = 1.0 * net[u][v].get(weightName, 1.0)
                eSum += w_uv / outDeg[u] * prevVisitProb[u]

            currVisitProb[v] = q * incomingTeleProb[v] + \
                (1 - q) * (eSum + zSum)
            maxDiff = max(maxDiff, abs(
                (prevVisitProb[v] - currVisitProb[v]) / currVisitProb[v]))

        if verbose:
            print('\tIteration %d, max difference %f' % (iters, maxDiff))
            if maxDiff < eps:
                print('PageRank converged after %d iterations, max difference %f.' % (
                    iters, maxDiff))

        if iters >= maxIters:
            print(
                'WARNING: PageRank terminated because max iters (%d) was reached.' % (maxIters))

        finished = (maxDiff < eps) or (iters >= maxIters)

    return currVisitProb

In [9]:
import networkx as nx

# Convert DGL graph to NetworkX
nx_graph = graph.to_networkx()

# Run PageRank
pagerank_scores = pagerank(nx_graph)


# Print pagerank scores (optional)
print(pagerank_scores)

{0: 0.00030057108506161706, 1: 0.00030352979162272763, 2: 0.00030057108506161706, 3: 0.0002912904249459003, 4: 0.00030057108506161706, 5: 0.0002410780858205541, 6: 0.00030057108506161706, 7: 0.00030057108506161706, 8: 0.0002666599291077922, 9: 0.00026802439198520396, 10: 0.0002488138554826537, 11: 0.00030057108506161706, 12: 0.0005173867036981069, 13: 0.00031858876334391593, 14: 0.00030057108506161706, 15: 0.00030057108506161706, 16: 0.000357590571799826, 17: 0.00031552080407086795, 18: 0.00028878618097678203, 19: 0.0003425582674339385, 20: 0.0002455296590146975, 21: 0.00027745013944704814, 22: 0.0003322913855054526, 23: 0.00026746936505631373, 24: 0.0002804630885928788, 25: 0.0002595402086676018, 26: 0.00024524743881051893, 27: 0.0003083079319718242, 28: 0.0005685895669589364, 29: 0.00027543989283235995, 30: 0.0002498901281896092, 31: 0.00024031607251620728, 32: 0.00025954435037082654, 33: 0.0003978718743727025, 34: 0.0002546098717428622, 35: 0.00030057108506161706, 36: 0.000300571085

In [10]:
import torch as th
import dgl.sparse as dglsp

# Get node features from the graph
features = graph.ndata["feat"]

# Convert PageRank scores to a tensor and match DGL node indices with NetworkX nodes
pagerank_tensor = th.tensor(
    [pagerank_scores[int(node.item())] for node in graph.nodes()]
)

# Normalize PageRank scores (optional step)
# pagerank_tensor /= pagerank_tensor.sum()

print("PageRank Tensor: ", pagerank_tensor)


# Get the shapes of the original features and PageRank tensor
print("Original Features Shape: ", features.shape)
print("PageRank Tensor Shape (before unsqueeze): ", pagerank_tensor.shape)

# Convert PageRank tensor to column vector
pagerank_tensor = pagerank_tensor.unsqueeze(1)
print("PageRank Tensor Shape (after unsqueeze): ", pagerank_tensor.shape)

# Ensure that the number of nodes matches between the features and PageRank tensor
if features.shape[0] == pagerank_tensor.shape[0]:
    # Concatenate PageRank scores with original node features
    combined_features = th.cat([features, pagerank_tensor], dim=1)
    print("Combined Features Shape: ", combined_features.shape)

    # Update the node features in the DGL graph
    graph.ndata["feat"] = combined_features
else:
    raise ValueError(
        "Mismatch in the number of nodes between features and PageRank tensor.")


# Option 2: Use PageRank scores as the sole node feature (if you want to replace the existing features)
# combined_features = pagerank_tensor.unsqueeze(1)  # If PageRank scores should be the only feature

PageRank Tensor:  tensor([0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0002])
Original Features Shape:  torch.Size([3327, 3703])
PageRank Tensor Shape (before unsqueeze):  torch.Size([3327])
PageRank Tensor Shape (after unsqueeze):  torch.Size([3327, 1])
Combined Features Shape:  torch.Size([3327, 3704])


In [11]:
print("Updated node features shape: ", graph.ndata["feat"].shape)

Updated node features shape:  torch.Size([3327, 3704])


In [12]:

class OGC(nn.Module):
    def __init__(self, graph):
        super(OGC, self).__init__()
        # Initialize the classifier
        self.linear_clf = LinearNeuralNetwork(
            nfeat=graph.ndata["feat"].shape[1],  # Feature dimension
            nclass=graph.ndata["label"].max().item() + 1,  # Number of classes
            bias=False,
        )

        self.label = graph.ndata["label"]  # Node labels
        self.label_one_hot = F.one_hot(
            graph.ndata["label"]).float()  # One-hot encoded labels

        # LIM trick: Diagonal matrix based on the training mask
        self.label_idx_mat = dglsp.diag(graph.ndata["train_mask"]).float()

        self.test_mask = graph.ndata["test_mask"]
        self.tv_mask = graph.ndata["train_mask"] + graph.ndata["val_mask"]

    def forward(self, x):
        return self.linear_clf(x)  # Forward pass through the linear classifier

    def update_embeds(self, embeds, pagerank_tensor, args):
        """
        Update embeddings using the PageRank-based approach.
        """
        # Predict labels using the current embeddings
        pred_label = self(embeds).data
        clf_weight = self.linear_clf.W.weight.data  # Classifier weights

        # Update the smoothness loss via PageRank scores
        embeds = embeds * pagerank_tensor.to(embeds.device)

        # Update the supervised loss via SEB (Supervised Embedding Backpropagation)
        deriv_sup = 2 * dglsp.matmul(
            dglsp.spmm(self.label_idx_mat, -self.label_one_hot +
                       pred_label), clf_weight
        )
        embeds = embeds - args.lr_sup * deriv_sup  # Gradient update for supervised loss

        # Reduce learning rate for supervised learning based on decline rate
        args.lr_sup = args.lr_sup * args.decline

        return embeds

In [13]:
model = OGC(graph).to(args.device)

In [14]:
sum(p.numel() for p in model.parameters())

22224

In [15]:
def train(model, embeds, pagerank_tensor, args):
    patience = 0
    _, _, last_acc, last_output = model_test(model, embeds)

    tv_mask = model.tv_mask
    optimizer = optim.SGD(model.parameters(), lr=args.lr_clf)

    for i in range(64):
        model.train()

        # print("Shape of embeds at epoch {}: {}".format(i + 1, embeds.shape))

        # Forward pass
        output = model(embeds)

        # Compute loss
        loss_tv = F.mse_loss(
            output[tv_mask], model.label_one_hot[tv_mask], reduction="sum")

        # Backward pass and optimization
        optimizer.zero_grad()
        loss_tv.backward()
        optimizer.step()

        # Updating node embeds by PageRank scores and SEB jointly
        embeds = model.update_embeds(embeds, pagerank_tensor, args)

        # Re-test the model with updated embeddings
        loss_tv, acc_tv, acc_test, pred = model_test(model, embeds)
        print(
            "epoch {} loss_tv {:.4f} acc_tv {:.4f} acc_test {:.4f}".format(
                i + 1, loss_tv, acc_tv, acc_test
            )
        )

        # Check for early stopping condition based on prediction similarity
        sim_rate = float(int((pred == last_output).sum()) / int(pred.shape[0]))
        if sim_rate > args.max_sim_rate:
            patience += 1
            if patience > args.max_patience:
                break

        last_acc = acc_test
        last_output = pred

    return last_acc

In [16]:
start_time = time.time()
res = train(model, combined_features, pagerank_tensor, args)
time_tot = time.time() - start_time

print(f"Test Acc:{res:.4f}")
print(f"Total Time:{time_tot:.4f}")

epoch 1 loss_tv 0.1655 acc_tv 0.8339 acc_test 0.6390
epoch 2 loss_tv 0.1639 acc_tv 0.8016 acc_test 0.6390
epoch 3 loss_tv 0.1643 acc_tv 0.8016 acc_test 0.6420
epoch 4 loss_tv 0.1643 acc_tv 0.8016 acc_test 0.6420
epoch 5 loss_tv 0.1645 acc_tv 0.8016 acc_test 0.6430
epoch 6 loss_tv 0.1646 acc_tv 0.8016 acc_test 0.6440
Test Acc:0.6430
Total Time:2.3197
